# scikit-learn Classification with  k-nearest Neighbors classification

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


#### Key points from the previous notebooks:
- It is ideal to explore and visualize the data prior to doing actual machine learning.
- A model must be instantiated, trained using `.fit` and used to predict using `.predict`.
- You must train your model and test it on separate data. `train_test_split` is a function used for this purpose.
- Metrics for evaluation exist, like accuracy, sensitivity and specificity. More nuances method like the confusion matrix and AUC (not covered yet) are also very useful. `classification_report` gives a neat summary.


In [ ]:
url = "https://raw.githubusercontent.com/thilinib/CBM101/main/E_Macine_Learning/data/preprocessed_diabetes.csv"
df = pd.read_csv(url)

In [ ]:
df.head()

In [ ]:
X = df.drop(columns=['Outcome'])
y = df['Outcome']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape


# K-Nearest neighbors

In [ ]:
clf = neighbors.KNeighborsClassifier(n_neighbors=5) # define the model and specify parameters

In [ ]:
clf.fit(X_train, y_train) # train the model

Just like that the model is trained, in two lines of code. Now you are left with a trained algorithm `clf`, which you can use to predict the group belonging of new samples.  

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Predictions
y_pred = clf.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Recall
recall = recall_score(y_test, y_pred, average='binary')

# Precision
precision = precision_score(y_test, y_pred, average='binary')

# F1 Score
f1 = f1_score(y_test, y_pred, average='binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


## Feature scaling

So far we have done minimal preprocessing. With few exceptions, you should always do some basic preprocessing, like standardizing each input feature to zero mean and unit variance. 

In addition to the `fit` method, many `sklearn` objects have a `transform` method, which lets you transform your dataset. In this case, the transform yields the dataset post normalization. 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
scaler = StandardScaler()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler.fit(X_train) # fit only the training set
X_train_scaled = scaler.transform(X_train) #transform the training dataset
X_scaled = scaler.transform(X) # For visualization purposes, let's also transform the full dataset (X)

# Plot before and after normalization
fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)  # Make two plots on the same row
ax[0].scatter(X.iloc[:, 0], X.iloc[:, 1], c=y)  # Use iloc to select the first two columns of X
ax[0].set_title('Before normalizing')  # Set the title of the first subplot
ax[1].scatter(X_scaled[:, 0], X_scaled[:, 1], c=y)  # Use the transformed X
ax[1].set_title('After normalizing')  # Set the title of the second subplot
plt.show()

NOTE:  you have to fit the scaler to the training set (not the whole dataset, X), and then transform `X_test` and `X_train` later.

In [ ]:
#load data again in to X and y
X = df.drop(columns=['Outcome'])
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)#split to training and test
scaler.fit(X_train) ## Fit the scaler on the training set

## Transform both the training and testing set
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#adding power transformation to training and testing set
pt = PowerTransformer()
X_train = pt.fit_transform(X_train)
X_test = pt.fit_transform(X_test)

# train the model with cross validation (using gridsearch)
clf = neighbors.KNeighborsClassifier(n_neighbors=5)

parameters = {}
clf_cv = GridSearchCV(clf, cv = 10, param_grid =parameters)
clf_cv.fit(X_train, y_train)


You can tune the model using GridSearchCV which will help you to find the best parameters of the model. Also it helps you to do cross validation at the same time.
[link](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [ ]:
# Predictions
y_pred = clf_cv.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,  recall_score, precision_score, f1_score

# Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Recall
recall = recall_score(y_test, y_pred, average='binary')

# Precision
precision = precision_score(y_test, y_pred, average='binary')

# F1 Score
f1 = f1_score(y_test, y_pred, average='binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


# Common pitfalls and recommended practices

1. Inconsistent preprocessing
    - Instead of passing the non-transformed X_test to predict, we should transform the test data, the same way we transformed the training data:
2. Data leakage
    - Data leakage occurs when information that would not be available at prediction time is used when building the model. The general rule is to never call fit on the test data
    - Although both train and test data subsets should receive the same preprocessing transformation (as described in the previous section), it is important that these transformations are only learnt from the training data. 
3. Controlling randomness
    - Not controlling randomness, leading to non-reproducible results. Use random_state parameter to control the randomness for reproducibility.